In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

import xgboost as xgb
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin

In [2]:
dados = pd.read_csv("dados_pre_processados.csv")
dados.drop("Unnamed: 0", axis = "columns", inplace=True)
dados.head()

,CodigoReceita,Volume,TipoLancamento,DataEmissaoNotaFiscal,CodigoCliente,CodigoMotorista,TempoPermanenciaObra,ConsumoTotal,Idade,TipoTraco,...,FCK,Rompimento7Dias,Rompimento28Dias,ConsumoPorM3,DesvioAplicado,ResultadoProjetado,PercentualCrescimentoReal,PercentualCrescimentoProjetado,RelacaoAguaCimento,DataProjecao
0,672051,8.0,B,2015-01-16 07:35:31.770,6248051,11080,02:41:00,2.720.000,28,N,...,30.0,20.83,31.81,340.0,3.5,33.321754,52.712434,59.970014,0.535294,2015-01-16
1,672051,8.0,B,2015-01-16 08:20:03.153,6248051,87079,02:46:00,2.720.000,28,N,...,30.0,19.62,31.21,340.0,3.5,31.799067,59.072375,62.074756,0.535294,2015-01-16
2,672051,7.0,B,2015-01-16 10:05:20.293,6248051,43080,01:32:00,2.380.000,28,N,...,30.0,21.97,36.68,340.0,3.5,34.733698,66.954939,58.096029,0.535294,2015-01-16
3,672051,8.0,B,2015-01-20 18:52:57.373,12343051,43080,01:10:00,2.720.000,28,N,...,30.0,28.21,40.21,340.0,3.5,42.118539,42.538107,49.303577,0.535294,2015-01-23
4,672051,6.0,B,2015-01-21 07:08:54.847,6248051,42080,00:35:00,2.040.000,28,N,...,30.0,25.80,38.74,340.0,3.5,39.330637,50.155039,52.444329,0.535294,2015-01-23


In [3]:
mask_aglomerante = dados["CodigoAglomerante"] == 75051
dados_aglomerante = dados[mask_aglomerante]
dados_aglomerante.head()

,CodigoReceita,Volume,TipoLancamento,DataEmissaoNotaFiscal,CodigoCliente,CodigoMotorista,TempoPermanenciaObra,ConsumoTotal,Idade,TipoTraco,...,FCK,Rompimento7Dias,Rompimento28Dias,ConsumoPorM3,DesvioAplicado,ResultadoProjetado,PercentualCrescimentoReal,PercentualCrescimentoProjetado,RelacaoAguaCimento,DataProjecao
38008,33488051,6.0,B,2017-03-08 14:43:53.140,133258051,812051,00:54:00,2.172.000,28,N,...,30.0,25.73,35.08,362.0,6.0,35.882105,36.338904,39.456298,0.538674,2017-03-10
38066,33535051,8.0,B,2017-11-13 08:06:27.303,225643998,85079,01:05:00,2.560.000,28,N,...,25.0,24.26,33.40,320.0,6.0,28.834552,37.675185,18.856358,0.559375,2017-11-17
38067,33535051,8.0,B,2017-11-21 07:35:58.153,225643998,624051,01:39:00,2.560.000,28,N,...,25.0,24.24,31.72,320.0,6.0,28.817159,30.858086,18.882667,0.678125,2017-11-24
38068,33535051,8.0,B,2017-12-11 08:50:55.937,225643998,731051,01:09:00,2.560.000,28,N,...,25.0,23.45,31.12,320.0,6.0,28.125844,32.707889,19.939631,0.678125,2017-12-15
38069,33535051,8.0,B,2017-12-12 08:02:35.123,225643998,340051,01:28:00,2.560.000,28,N,...,25.0,20.88,28.39,320.0,6.0,25.816563,35.967433,23.642543,0.678125,2017-12-15


In [4]:
len(dados_aglomerante)

466

In [11]:
dados_aglomerante["LogRompimento7Dias"] = np.log(dados_aglomerante["Rompimento7Dias"])

<ipython-input-11-336d0659c33b>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [12]:
fig = px.box(dados_aglomerante, x="TipoLancamento", y="PercentualCrescimentoReal", title='Tipo de Lançamento x Rompimento 28 Dias')
fig.show()

In [13]:
fig = px.box(dados_aglomerante, x="TipoTraco", y="PercentualCrescimentoReal", title='Tipo de Traço x Rompimento 28 Dias')
fig.show()

In [14]:
fig = px.box(dados_aglomerante, x="FCK", y="PercentualCrescimentoReal", title='FCK x Rompimento 28 Dias')
fig.show()

In [16]:
fig = px.scatter(dados_aglomerante, x="LogRompimento7Dias", y="Rompimento28Dias", title='FCK x Rompimento 28 Dias')
fig.show()

In [17]:
dados_aglomerante["TipoLancamento"].replace('C', 0, inplace = True)
dados_aglomerante["TipoLancamento"].replace('B', 1, inplace = True)

dados_aglomerante["TipoTraco"].replace('N', 0, inplace = True)
dados_aglomerante["TipoTraco"].replace('E', 1, inplace = True)

/home/marina/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:4563: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [18]:
class TypeSelector(BaseEstimator, TransformerMixin):
    def __init__(self, dtype):
        self.dtype = dtype
    def fit(self, X, y=None):
        return self
    def transform(self, X):        
        assert isinstance(X, pd.DataFrame)
        return X.select_dtypes(include=[self.dtype])
    
class Debug(BaseEstimator, TransformerMixin):
    def __init__(self, dtype):
        self.dtype = dtype
    def fit(self, X, y=None):
        return self
    def transform(self, X):        
        print(X)

modelo = Pipeline([
  #('features', FeatureUnion(
  #    transformer_list=[
  #        ('numericals', Pipeline([
  #            ('scaler', MinMaxScaler())
  #        ]))
  #    ]
  #)),
  ('model', GradientBoostingRegressor(random_state = 1))
  #('model', DecisionTreeRegressor())
  #('model', xgb.XGBRegressor())
])

In [19]:
dados_aglomerante.sort_values("DataProjecao", inplace=True)

<ipython-input-19-d0dafc5bc8fe>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [20]:
dados_aglomerante.head(30)

,CodigoReceita,Volume,TipoLancamento,DataEmissaoNotaFiscal,CodigoCliente,CodigoMotorista,TempoPermanenciaObra,ConsumoTotal,Idade,TipoTraco,...,Rompimento7Dias,Rompimento28Dias,ConsumoPorM3,DesvioAplicado,ResultadoProjetado,PercentualCrescimentoReal,PercentualCrescimentoProjetado,RelacaoAguaCimento,DataProjecao,LogRompimento7Dias
38008,33488051,6.0,1,2017-03-08 14:43:53.140,133258051,812051,00:54:00,2.172.000,28,0,...,25.73,35.08,362.0,6.0,35.882105,36.338904,39.456298,0.538674,2017-03-10,3.247658
38107,33541051,6.0,1,2017-03-24 13:21:38.870,12560051,94079,00:30:00,2.124.000,28,0,...,22.32,31.90,354.0,6.0,31.821606,42.921147,42.569919,0.528249,2017-03-24,3.105483
38106,33541051,6.0,1,2017-03-24 12:13:31.350,12560051,173051,00:18:00,2.124.000,28,0,...,23.26,30.93,354.0,6.0,32.951628,32.975064,41.666500,0.533898,2017-03-24,3.146735
38113,33541051,6.0,1,2017-03-31 13:04:26.390,12560051,66079,01:45:00,2.124.000,28,0,...,22.01,30.60,354.0,6.0,31.447056,39.027715,42.876218,0.539548,2017-03-31,3.091497
38111,33541051,7.0,1,2017-03-29 10:28:03.440,12560051,704051,00:23:00,2.478.000,28,0,...,19.91,30.30,354.0,6.0,28.883882,52.184832,45.072236,0.533898,2017-03-31,2.991222
38110,33541051,7.0,1,2017-03-29 09:19:57.647,12560051,727051,00:22:00,2.478.000,28,0,...,23.24,31.02,354.0,6.0,32.927673,33.476764,41.685339,0.533898,2017-03-31,3.145875
38109,33541051,7.0,1,2017-03-29 08:27:14.300,12560051,704051,00:21:00,2.478.000,28,0,...,22.74,30.83,354.0,6.0,32.327560,35.576077,42.161652,0.533898,2017-03-31,3.124125
38108,33541051,7.0,1,2017-03-29 07:34:41.580,12560051,66079,00:32:00,2.478.000,28,0,...,23.09,31.30,354.0,6.0,32.747889,35.556518,41.827148,0.533898,2017-03-31,3.139400
38112,33541051,7.0,1,2017-03-29 11:22:00.967,12560051,731051,01:20:00,2.478.000,28,0,...,21.95,30.60,354.0,6.0,31.374452,39.407745,42.936000,0.533898,2017-03-31,3.088767
38428,33840051,8.0,1,2017-04-13 08:20:28.960,137335051,517051,00:15:00,2.984.000,28,0,...,23.60,32.41,373.0,6.0,33.358292,37.330508,41.348697,0.493298,2017-04-14,3.161247


In [41]:
dados_aglomerante_modelo = dados_aglomerante[["LogRompimento7Dias", "Rompimento28Dias", "ResultadoProjetado", "DataProjecao"]]

mask_data_projecao_treino_inicio = dados_aglomerante_modelo["DataProjecao"] >= '2017-04-14'
mask_data_projecao_treino_final = dados_aglomerante_modelo["DataProjecao"] <= '2017-05-12'
mask_data_projecao_teste = dados_aglomerante_modelo["DataProjecao"] == '2017-05-19'

dados_treino = dados_aglomerante_modelo[mask_data_projecao_treino_final]
#dados_treino = dados_aglomerante_modelo[mask_data_projecao_treino_inicio & mask_data_projecao_treino_final]

dados_teste = dados_aglomerante_modelo[mask_data_projecao_teste]

In [42]:
x_train = dados_treino[["LogRompimento7Dias", "ResultadoProjetado"]]
y_train = dados_treino["Rompimento28Dias"].values

x_test = dados_teste[["LogRompimento7Dias", "ResultadoProjetado"]]
y_test = dados_teste["Rompimento28Dias"].values

In [43]:
y_train = y_train.reshape(-1, 1)
#x_train = x_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)
#x_test = x_test.reshape(-1, 1)

In [44]:
x_train_modelo = x_train["LogRompimento7Dias"].values
x_test_modelo = x_test["LogRompimento7Dias"].values

x_train_modelo = x_train_modelo.reshape(-1,1)
x_test_modelo = x_test_modelo.reshape(-1,1)

In [45]:
modelo.fit(x_train_modelo, y_train)
y_pred = modelo.predict(x_test_modelo)

/home/marina/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



In [46]:
df = pd.DataFrame(columns = ["Rompimento28Dias", "ResultadoProjetado", "ResultadoPredito"])

df["Rompimento28Dias"] = y_test.reshape(1, -1)[0]
df["ResultadoProjetado"] = x_test["ResultadoProjetado"].values
df["ResultadoPredito"] = y_pred

In [47]:
df["ErroProjetado"] = abs(df["ResultadoProjetado"] - df["Rompimento28Dias"])
df["ErroPredito"] = abs(df["ResultadoPredito"] - df["Rompimento28Dias"])

In [48]:
df

,Rompimento28Dias,ResultadoProjetado,ResultadoPredito,ErroProjetado,ErroPredito
0,46.74,49.191669,42.995033,2.451669,3.744967
1,42.62,46.816616,45.365636,4.196616,2.745636
2,30.02,30.865215,30.601968,0.845215,0.581968
3,46.78,48.951039,42.995033,2.171039,3.784967
4,28.11,27.745969,25.397994,0.364031,2.712006
5,46.60,44.960607,44.582199,1.639393,2.017801
6,46.46,48.436012,42.995033,1.976012,3.464967
7,52.00,53.626093,51.233159,1.626093,0.766841
8,49.79,49.399258,49.043259,0.390742,0.746741
9,51.63,49.333726,42.995033,2.296274,8.634967


In [55]:
fig = px.scatter(df, y=["ResultadoPredito", "Rompimento28Dias", "ResultadoProjetado"], title='FCK x Rompimento 28 Dias')
fig.show()

In [49]:
df["ErroProjetado"].sum()

17.957083810956085

In [50]:
df["ErroPredito"].sum()

29.200858923622526

In [51]:
mask_modelo_predito_melhor = df["ErroPredito"] < df["ErroProjetado"]
quantidade_nf_modelo_eh_melhor = len(df[mask_modelo_predito_melhor])
quantidade_total = len(df)

quantidade_nf_modelo_eh_melhor/quantidade_total

0.3

In [ ]:
0.6493506493506493  - 0,22 - 12 - GradientBoostingRegressor